In [1]:
from IPython.core.display import display, HTML
import math
import pandas as pd

import mlos.global_values as global_values
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Spaces import ContinuousDimension, Point, SimpleHypergrid
from mlos.Tracer import Tracer

display(HTML("<style>.container { width:100% !important; }</style>"))

global_values.declare_singletons()
global_values.tracer = Tracer(actor_id="MetaOptimizer", thread_id=0)

# Let's stand up the Optimizer Microservice
#
optimizer_factory = BayesianOptimizerFactory()

In [2]:
# Let's make a meta optimizer.
#
meta_optimizer_config = bayesian_optimizer_config_store.get_config_by_name("default_with_glow_worm")
meta_optimizer_config.homogeneous_random_forest_regression_model_config.n_estimators = 100
meta_optimizer_config.homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.n_new_samples_before_refit = 1
meta_optimizer_config.experiment_designer_config.fraction_random_suggestions = 0.2

meta_optimizer = optimizer_factory.create_local_optimizer(
    optimizer_config=meta_optimizer_config,
    optimization_problem=OptimizationProblem(
        parameter_space=bayesian_optimizer_config_store.parameter_space,
        objective_space=SimpleHypergrid(
            name="predictions",
            dimensions=[
                ContinuousDimension(name="optimum_value_after_100_iterations", min=-math.inf, max=math.inf)
            ]
        ),
        objectives=[Objective(name='optimum_value_after_100_iterations', minimize=True)]
    )
)

09/29/2020 21:30:50 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  40 -    create_local_optimizer() ] Creating a bayesian optimizer with config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 100,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_mo

In [3]:
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.ObjectiveFunctionConfigStore import objective_function_config_store
objective_function_config = objective_function_config_store.get_config_by_name('three_level_quadratic')
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)

print(objective_function_config.to_json(indent=2))

{
  "implementation": "ThreeLevelQuadratic"
}


In [4]:
print(objective_function.parameter_space)

  Name: three_level_quadratic_config
  Dimensions:
    vertex_height: {low, 5, 15}

  IF vertex_height IN {5} THEN (
    Name: medium_quadratic_params
    Dimensions:
      x_1: [-100.00, 100.00]
      x_2: [-100.00, 100.00]
  )

  IF vertex_height IN {15} THEN (
    Name: high_quadratic_params
    Dimensions:
      x_1: [-100.00, 100.00]
      x_2: [-100.00, 100.00]
  )

  IF vertex_height IN {low} THEN (
    Name: low_quadratic_params
    Dimensions:
      x_1: [-100.00, 100.00]
      x_2: [-100.00, 100.00]
  )


In [5]:
from mlos.Logger import create_logger
from mlos.Optimizers.RegressionModels.RegressionModelFitState import RegressionModelFitState

# Let us set up the lists to track optima over time.
#
best_observation_num_observations = []
best_observation_configs = []
best_observations = []

predicted_value_num_observations = []
best_predicted_value_configs = []
best_predicted_values = []

regression_model_fit_state = RegressionModelFitState()

In [6]:
import concurrent.futures
from mlos.Optimizers.OptimumDefinition import OptimumDefinition
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import GoodnessOfFitMetrics, DataSetType
from mlos.OptimizerEvaluationTools.ParallelOptimization import run_optimization

logger = create_logger("Optimizing.")

num_desired_runs = 50
num_completed_runs = 0
max_concurrent_jobs = 7

with concurrent.futures.ProcessPoolExecutor(max_workers=max_concurrent_jobs) as executor:
    outstanding_futures = set()
    while num_completed_runs < num_desired_runs:
        logger.info(f"[{num_completed_runs}/{num_desired_runs}]")

        # Let's keep submitting new jobs to the pool until we have the desired number of executions.
        #
        num_remaining_jobs_to_schedule = min(num_desired_runs - num_completed_runs - len(outstanding_futures), max_concurrent_jobs)
        if num_remaining_jobs_to_schedule > 0:
            for _ in range(num_remaining_jobs_to_schedule):
                logger.info("Getting suggestion from meta optimizer.")
                inner_optimizer_config = meta_optimizer.suggest()
                logger.info(f"Inner optimizer config: {inner_optimizer_config.to_json(indent=2)}")
                future = executor.submit(
                    run_optimization,
                    inner_optimizer_config.to_json(),
                    objective_function_config.to_json(),
                    num_iterations=10
                )
                outstanding_futures.add(future)

        # Now let's wait for any future to complete. 
        #
        logger.info("Waiting for futures to complete.")
        done_futures, outstanding_futures = concurrent.futures.wait(outstanding_futures, return_when=concurrent.futures.FIRST_COMPLETED)
        logger.info(f"{len(done_futures)} future(s) completed.")
        for future in done_futures:
            inner_optimizer_config_json, best_observation_value_json = future.result()
            inner_optimizer_config = Point.from_json(inner_optimizer_config_json)
            best_observation_value = Point.from_json(best_observation_value_json)
            logger.info(f"Registering {best_observation_value_json} with meta optimizer.")
            meta_optimizer.register(
                feature_values_pandas_frame=inner_optimizer_config.to_dataframe(),
                target_values_pandas_frame=Point(optimum_value_after_100_iterations=best_observation_value.y).to_dataframe()
            )
            logger.info(f"Registered observation with meta optimizer.")
            num_completed_runs += 1


09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  15 -                  <module>() ] [0/50]
09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 4,
  "homogeneous_random_forest_regression_model_config.n_estimators": 34,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.7225566612631158,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.9661020384896781,
  "homogeneous_random_forest_regression_model_conf

09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 48,
  "homogeneous_random_forest_regression_model_config.n_estimators": 3,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.5402679085535953,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.7897391257557762,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 0,
  "homog

09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:39 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 54,
  "homogeneous_random_forest_regression_model_config.n_estimators": 70,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.35028160641122175,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.5250333049146647,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 0,
  "hom

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 79,
  "homogeneous_random_forest_regression_model_config.n_estimators": 124,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.20423892109968667,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.8608050802252591,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 0,
  "ho

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 12,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.6245834482488599,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.4600504687285377,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 1,
  "homo

...

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  47 -                  <module>() ] Registered observation with meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  42 -                  <module>() ] Registering {"y": 567.751416613019} with meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  47 -                  <module>() ] Registered observation with meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  15 -                  <module>() ] [7/50]
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 12,
  "homogeneous_random_forest_regression_model_config.n_estimators": 57,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.31632678104809997,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.0070533268300593255,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 1,
  "

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  22 -                  <module>() ] Getting suggestion from meta optimizer.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  24 -                  <module>() ] Inner optimizer config: {
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 69,
  "homogeneous_random_forest_regression_model_config.n_estimators": 106,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.5670877572601244,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.3609655657791033,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.bootstrap": 1,
  "hom

09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  37 -                  <module>() ] 1 future(s) completed.
09/29/2020 21:31:42 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  42 -                  <module>() ] Registering {"y": 819.614454692435} with meta optimizer.
09/29/2020 21:31:43 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  47 -                  <module>() ] Registered observation with meta optimizer.
09/29/2020 21:31:43 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  15 -                  <module>() ] [10/50]
09/29/2020 21:31:43 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  35 -                  <module>() ] Waiting for futures to complete.
09/29/2020 21:31:43 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  37 -                  <module>() ] 26 future(s) completed.
09/29/2020 21:31:43 -          

09/29/2020 21:31:50 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  42 -                  <module>() ] Registering {"y": 1787.9743343270525} with meta optimizer.
09/29/2020 21:31:51 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  47 -                  <module>() ] Registered observation with meta optimizer.
09/29/2020 21:31:51 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  42 -                  <module>() ] Registering {"y": 1317.805232512107} with meta optimizer.
09/29/2020 21:31:51 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  47 -                  <module>() ] Registered observation with meta optimizer.
09/29/2020 21:31:51 -                Optimizing. -    INFO - [<ipython-input-6-447593bc75bd>:  42 -                  <module>() ] Registering {"y": 1163.6311926996073} with meta optimizer.
09/29/2020 21:31:51 -                Optimizing. -    INFO - [<ipython-input-6-447593b

In [ ]:
logger = create_logger("Optimizing.")
outer_loop_iteration = 0

In [ ]:
from mlos.Optimizers.OptimumDefinition import OptimumDefinition
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import GoodnessOfFitMetrics, DataSetType



start_iteration_num = outer_loop_iteration
end_iteration_num = start_iteration_num + 50

for outer_loop_iteration in range(start_iteration_num, end_iteration_num):
    inner_optimizer_config = meta_optimizer.suggest()
    
    inner_optimizer = optimizer_factory.create_local_optimizer(
        optimizer_config=inner_optimizer_config,
        optimization_problem=OptimizationProblem(
            parameter_space=objective_function.parameter_space,
            objective_space=objective_function.output_space,
            objectives=[Objective(name='y', minimize=True)]
        )
    )
    
    ############################################# INNER OPTIMIZATION #############################################
    
    try:
        for i in range(100):
            parameters = inner_optimizer.suggest()
            objectives = objective_function.evaluate_point(parameters)
            logger.info(f"[{i+1}/{100}]Parameters: {parameters}, objectives: {objectives}")
            inner_optimizer.register(feature_values_pandas_frame=parameters.to_dataframe(), target_values_pandas_frame=objectives.to_dataframe())
        best_observation_config, best_observation = inner_optimizer.optimum(OptimumDefinition.BEST_OBSERVATION)
        
        logger.info(f"[{outer_loop_iteration+1}/{end_iteration_num}] {best_observation}")
        meta_optimizer.register(feature_values_pandas_frame=inner_optimizer_config.to_dataframe(), target_values_pandas_frame=Point(optimum_value_after_100_iterations=best_observation.y).to_dataframe())
    
    except:
        logger.error("Failed to complete inner optimization.", exc_info=True)
    
    ############################################# INNER OPTIMIZATION #############################################
    
    
    if meta_optimizer.trained:
        gof_metrics = meta_optimizer.compute_surrogate_model_goodness_of_fit()
        regression_model_fit_state.set_gof_metrics(data_set_type=DataSetType.TRAIN, gof_metrics=gof_metrics)
        
    best_observation_num_observations.append(outer_loop_iteration)
    
    best_observation_config, best_observation = meta_optimizer.optimum(OptimumDefinition.BEST_OBSERVATION)    
    best_observation_configs.append(best_observation_config)
    best_observations.append(best_observation)
    
    try:
        best_predicted_value_config, best_predicted_value = meta_optimizer.optimum(OptimumDefinition.PREDICTED_VALUE_FOR_OBSERVED_CONFIG)
        best_predicted_value_configs.append(best_predicted_value_config)
        best_predicted_values.append(best_predicted_value)
        predicted_value_num_observations.append(outer_loop_iteration)
    except:
        pass
    

In [ ]:
best_observation_config, best_observation

In [ ]:
best_predicted_value_config, best_predicted_value

In [ ]:
# Best observation dataframe
#
best_observation_df = pd.DataFrame([observation.to_dict() for observation in best_observations])
best_observation_df['num_observations'] = best_observation_num_observations
best_observation_df = pd.concat([best_observation_df.drop_duplicates(subset=['optimum_value_after_100_iterations'], keep='last'), best_observation_df.drop_duplicates(subset=['optimum_value_after_100_iterations'], keep='first')]).sort_index()
best_observation_df

In [ ]:
best_predicted_value_df = pd.DataFrame([predicted_value.to_dict() for predicted_value in best_predicted_values])
best_predicted_value_df['num_observations'] = predicted_value_num_observations
best_predicted_value_df = pd.concat([best_predicted_value_df.drop_duplicates(subset=['predicted_value'], keep='last'), best_predicted_value_df.drop_duplicates(subset=['predicted_value'], keep='first')]).sort_index()
best_predicted_value_df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

fig, axs = plt.subplots(1, figsize=(11, 20), dpi=80, sharex=True)

axs.plot(best_observation_df['num_observations'], best_observation_df['optimum_value_after_100_iterations'], label='optimum_value_after_100_iterations')
axs.plot(best_predicted_value_df['num_observations'], best_predicted_value_df['predicted_value'], label='predicted_value')
axs.set_ylabel('y')
axs.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
axs.set_xticks(best_observation_df['num_observations'][::2])
axs.grid(True)
axs.set_xlabel('num_observations')
axs.legend()  
fig.show()

In [ ]:
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import DataSetType

# Let's take a look at goodness of fit data.
#
goodness_of_fit_dataframe = regression_model_fit_state.get_goodness_of_fit_dataframe(data_set_type=DataSetType.TRAIN) # TODO: add support to evaluate GoF on test data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

gof_df = goodness_of_fit_dataframe
columns_to_plot = [name for name in gof_df.columns.values if name not in ('observation_count', 'prediction_count', 'last_refit_iteration_number')]
num_plots = len(columns_to_plot)
fig, axs = plt.subplots(num_plots, figsize=(11, 20), dpi=80, sharex=True)

for i, column in enumerate(columns_to_plot):
    axs[i].plot(gof_df['last_refit_iteration_number'], gof_df[column], marker='o', label=column)
    axs[i].set_ylabel(column)
    axs[i].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
    axs[i].set_xticks(gof_df['last_refit_iteration_number'])
    axs[i].grid(True)
    if i == num_plots - 1:
        axs[i].set_xlabel('last_refit_iteration_number')
        
fig.show()

In [ ]:
global_values.tracer.dump_trace_to_file(r"E:\code\new_mlos\source\Mlos.Python\temp\meta_optimizer.json")